In [2]:
# Install Gradio for creating a simple web interface
# Install the latest version of Hugging Face Transformers (for GPT-2)
!pip install -q gradio
!pip install -q git+https://github.com/huggingface/transformers.git

  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [3]:
# Import Gradio for UI, TensorFlow backend for running GPT2, and the tokenizer/model from Transformers
import gradio as gr
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [4]:
# Load the GPT-2 tokenizer to convert input text into tokens
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load the GPT-2 model and assign the end-of-sequence token for padding
# This model will be used to generate text
model = TFGPT2LMHeadModel.from_pretrained(
    "gpt2",
    pad_token_id=tokenizer.eos_token_id  # Ensures the model handles padding correctly
)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

C:\Users\gmkai\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gmkai\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
# Define a function that accepts user input and generates a text paragraph
def generate_text(input_text):
    # Convert the input text into model-readable format (tensor of tokens)
    input_ids = tokenizer.encode(input_text, return_tensors='tf')

    # Generate new text using beam search (more coherent than greedy or random)
    beam_output = model.generate(
        input_ids,
        max_length=350,              # Maximum length of the generated text
        num_beams=5,                 # Beam search with 5 beams (better quality)
        no_repeat_ngram_size=2,     # Avoid repeating 2-word phrases
        early_stopping=True         # Stop when output seems complete
    )

    # Decode the generated output into human-readable text
    output = tokenizer.decode(
        beam_output[0],
        skip_special_tokens=True,   # Remove tokens like
        clean_up_tokenization_spaces=True  # Clean extra spaces
    )

    # Return only full sentences, cleaned up
    return ".".join(output.split(".")) + "."


In [5]:
# Define a textbox output component
output_text = gr.Textbox()

# Create and launch the Gradio interface
gr.Interface(
    fn=generate_text,         # Function to run on input
    inputs="textbox",         # Input component (user prompt)
    outputs=output_text,      # Output component (generated paragraph)
    title="🧠 GENERATIVE TEXT MODEL (GPT-2)"  # Interface title
).launch()


NameError: name 'generate_text' is not defined